In [9]:
#!pip install pandas
#!pip install openpyxl

import pandas as pd

df = pd.read_excel("/Users/xaviergarcia/Downloads/sprint10.xlsx", header=None)
df.columns = ['Codi','Nom', 'Cognoms', 'DNI',  "País d'origen",  'Ciutat',  'Dia de Naixement',  'Mes de Naixement', 'Any de Naixement',	'Gènere',	'Salari mensual',	'Fills'	,'No Fills',	'Grup Professional']
#df.columns = df.iloc[3]   # assignem la tercera fila com a noms
#df.rename(columns={None: "Codi"}, inplace=True)
df.drop(range(0,4), inplace=True)
df = df.reset_index(drop=True)
df_ordenat = df.sort_values(by=["País d'origen", 'Ciutat'], ascending=True)
print(df_ordenat)
print(df.head(10))
print('El DNI només té valors únics:', df["DNI"].is_unique)

    Codi      Nom              Cognoms        DNI País d'origen      Ciutat  \
21    21      Mia    Schneider Fischer  28973553Z      Alemanya      Berlín   
154  154    Laura    Schneider Fischer  37399141L      Alemanya      Berlín   
224  224      Lea  Schneider Schneider  37368317L      Alemanya      Berlín   
278  278      Mia              Fischer  21390098Z      Alemanya      Berlín   
602  602    Jonas            Schneider  44060014R      Alemanya      Berlín   
..   ...      ...                  ...        ...           ...         ...   
547  547    Emily         Taylor Jones  89577876S    Regne Unit  Manchester   
728  728   George          Brown Jones  57441590Y    Regne Unit  Manchester   
751  751   Olivia          Brown Brown  58204038A    Regne Unit  Manchester   
854  854     Isla          Jones Brown  28367234K    Regne Unit  Manchester   
872  872  William                Smith  82046699J    Regne Unit  Manchester   

    Dia de Naixement Mes de Naixement Any de Naixem

In [10]:
import numpy as np

df["Nom Complet"] = df["Nom"] + " " + df["Cognoms"]
df["Nascut a Espanya"] = df["País d'origen"] == "Espanya"
df = df.set_index("DNI")
df.rename(columns={
    "Dia de Naixement": "Dia",
    "Mes de Naixement": "Mes",
    "Any de Naixement": "Any"
}, inplace=True)
df["Gènere"] = df["Gènere"].replace({
    "H": "Home",
    "D": "Dona",
    "A": "Altres",
    "NC": np.nan   # dades faltants
})

df_per_mostrar = df[['Nom','Cognoms', 'Nom Complet', "País d'origen", "Nascut a Espanya", "Dia", "Mes", "Any", "Gènere"]]
print(df_per_mostrar.head(10))


               Nom           Cognoms              Nom Complet País d'origen  \
DNI                                                                           
16928694K     Inês    Ferreira Silva      Inês Ferreira Silva      Portugal   
27724652S    Clara  Sánchez Martínez   Clara Sánchez Martínez       Espanya   
38141675A   Fatima             Fassi             Fatima Fassi        Marroc   
59157262R  Khadija   Bennani Bennani  Khadija Bennani Bennani        Marroc   
69630528M     Toni    Sánchez García      Toni Sánchez García       Espanya   
26930927E     Laia   Castells Ferrer     Laia Castells Ferrer       Espanya   
78247593G    Carla            García             Carla García       Espanya   
21774231J    Marco             Rossi              Marco Rossi        Itàlia   
87316866D  Alessia             Rossi            Alessia Rossi        Itàlia   
14219925Z     Toni          Martínez            Toni Martínez       Espanya   

           Nascut a Espanya Dia Mes   Any  Gènere  

In [11]:
def combinar_fills(row):
    if row["Fills"] > 0:
        return "Sí"
    elif row["No Fills"] > 0:
        return "No"
    else:
        return pd.NA  
df["Fills"] = df.apply(combinar_fills, axis=1)
print(df)

          Codi      Nom           Cognoms País d'origen     Ciutat Dia Mes  \
DNI                                                                          
16928694K    0     Inês    Ferreira Silva      Portugal     Lisboa  25   2   
27724652S    1    Clara  Sánchez Martínez       Espanya  Barcelona  18   3   
38141675A    2   Fatima             Fassi        Marroc      Rabat   6  11   
59157262R    3  Khadija   Bennani Bennani        Marroc      Rabat  20   1   
69630528M    4     Toni    Sánchez García       Espanya  Barcelona   9   8   
...        ...      ...               ...           ...        ...  ..  ..   
25161375F  995    Marta     Ferrer Ferrer       Espanya    Sevilla   1   6   
52145541P  996     Joan            García       Espanya    Sevilla  11   4   
69760120X  997     Laia   Ferrer Martínez       Espanya  Barcelona  11  11   
82947791W  998    Jordi            García       Espanya  Barcelona  23   5   
89253307W  999    Clara     Sánchez López       Espanya     Bilb

In [12]:
#print(df["Salari mensual"].dtype)
df["Salari mensual"] = (
    df["Salari mensual"]
    .str.replace(".", "", regex=False) 
    .str.replace("€", "", regex=False) 
    .str.strip()
)
df["Salari mensual"] = pd.to_numeric(df["Salari mensual"], errors="coerce")
resum = df.groupby("Gènere")["Salari mensual"].agg(
    Sou_Mig="mean",
    Sou_Media="median",
    Sou_Minim="min",
    Sou_Maxim="max"
)
resum["Sou_Mig"] = resum["Sou_Mig"].round(2)
resum = resum.sort_values(by="Sou_Mig", ascending=False)
print(resum)

        Sou_Mig  Sou_Media  Sou_Minim  Sou_Maxim
Gènere                                          
Home    1643.25     1531.0        737       3356
Altres  1626.59     1545.0        703       3175
Dona    1469.44     1361.5        665       3021


In [13]:
#!pip install jinja2 matplotlib
import matplotlib.pyplot as plt
taula_resum = pd.pivot_table(
    df,
    values="Salari mensual",
    index="Gènere",
    columns="País d'origen",
    aggfunc="mean",
    margins=True,          
    margins_name="Mitjana"
)
taula_resum.style.background_gradient(cmap="Blues").format("{:.2f} €")


País d'origen,Alemanya,Argentina,Colòmbia,Espanya,França,Itàlia,Marroc,Mèxic,Portugal,Regne Unit,Mitjana
Gènere,,,,,,,,,,,
Altres,951.00 €,1141.00 €,1030.00 €,1706.18 €,nan €,1423.00 €,1365.00 €,1372.00 €,1765.00 €,1921.00 €,1626.59 €
Dona,1804.31 €,1291.80 €,1497.75 €,1460.16 €,1566.47 €,1247.18 €,1405.21 €,1517.80 €,1488.55 €,1489.46 €,1469.44 €
Home,2067.43 €,1583.29 €,1554.67 €,1682.11 €,1389.25 €,1672.88 €,1531.00 €,1625.00 €,1497.00 €,1162.56 €,1643.25 €
Mitjana,1851.38 €,1463.39 €,1495.54 €,1581.21 €,1462.73 €,1425.95 €,1447.33 €,1558.42 €,1523.33 €,1423.56 €,1560.99 €


In [14]:
import pandas as pd
from datetime import datetime

#df["DataNaixement"] = pd.to_datetime(df[["Any", "Mes", "Dia"]])
df["Data Naixement"] = pd.to_datetime(df["Any"].astype(str) + '-' + df["Mes"].astype(str) + '-' + df["Dia"].astype(str))

def calcular_edat(data_naixement):
    avui = pd.Timestamp.today()
    edat = avui.year - data_naixement.year
    if (avui.month, avui.day) < (data_naixement.month, data_naixement.day):
        edat -= 1
    return edat

df["Edat"] = df["Data Naixement"].apply(calcular_edat)

print(df)

          Codi      Nom           Cognoms País d'origen     Ciutat Dia Mes  \
DNI                                                                          
16928694K    0     Inês    Ferreira Silva      Portugal     Lisboa  25   2   
27724652S    1    Clara  Sánchez Martínez       Espanya  Barcelona  18   3   
38141675A    2   Fatima             Fassi        Marroc      Rabat   6  11   
59157262R    3  Khadija   Bennani Bennani        Marroc      Rabat  20   1   
69630528M    4     Toni    Sánchez García       Espanya  Barcelona   9   8   
...        ...      ...               ...           ...        ...  ..  ..   
25161375F  995    Marta     Ferrer Ferrer       Espanya    Sevilla   1   6   
52145541P  996     Joan            García       Espanya    Sevilla  11   4   
69760120X  997     Laia   Ferrer Martínez       Espanya  Barcelona  11  11   
82947791W  998    Jordi            García       Espanya  Barcelona  23   5   
89253307W  999    Clara     Sánchez López       Espanya     Bilb

In [15]:
df_increment = pd.DataFrame({
    "Grup Professional": ["Grup A","Grup B","Grup C","Grup D"],
    "Increment": ["5%","3,5%","2%","8%"]
})
df = df.merge(df_increment, on="Grup Professional")
df["Increment_num"] = df["Increment"].str.replace(",", ".").str.replace("%","").astype(float) / 100
df["Salari mensual"] = df["Salari mensual"] * (1 + df["Increment_num"])

print(df.head(10))

  Codi      Nom           Cognoms País d'origen     Ciutat Dia Mes   Any  \
0    0     Inês    Ferreira Silva      Portugal     Lisboa  25   2  1953   
1    1    Clara  Sánchez Martínez       Espanya  Barcelona  18   3  1996   
2    2   Fatima             Fassi        Marroc      Rabat   6  11  2005   
3    3  Khadija   Bennani Bennani        Marroc      Rabat  20   1  1995   
4    4     Toni    Sánchez García       Espanya  Barcelona   9   8  1999   
5    5     Laia   Castells Ferrer       Espanya     Madrid  16  12  1953   
6    6    Carla            García       Espanya   València  24  12  1957   
7    7    Marco             Rossi        Itàlia       Torí   3  11  2002   
8    8  Alessia             Rossi        Itàlia       Milà   6  12  1989   
9    9     Toni          Martínez       Espanya  Barcelona  16   5  1964   

   Gènere  Salari mensual Fills No Fills Grup Professional  \
0    Dona         1184.04    No     True            Grup B   
1    Dona         1315.65    Sí      Na

In [17]:
import pandas as pd

grups = df["Grup Professional"].unique()

for grup in grups:
    df_grup = df[df["Grup Professional"] == grup]
    nom_fitxer = f"dades_{grup.replace(' ', '')}.xlsx"
    df_grup.to_excel(nom_fitxer, index=False)

df_resum = df.groupby("Grup Professional").agg(
    Treballadors=("Grup Professional", "count"),
    Sou_mig=("Salari mensual", "mean"),
    Edat_mediana=("Edat", "median")
).reset_index()
df_resum.to_excel("resum_grups.xlsx", index=False)

In [50]:
#!pip install seaborn
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.api.types import is_numeric_dtype, is_datetime64_any_dtype

import os

def generar_grafics(df):

    os.makedirs("graficsS10", exist_ok=True)

    for col in df.columns:
        plt.figure(figsize=(12, 9))

        if is_numeric_dtype(df[col]):
            sns.histplot(df[col].dropna(), kde=True)
            plt.title(f"Histograma de {col}")
            tipus = "numeric"
        elif is_datetime64_any_dtype(df[col]):
            anys = df[col].dropna().dt.year.value_counts().sort_index()
            sns.barplot(x=anys.index, y=anys.values)
            plt.title(f"Distribució per anys de {col}")
            plt.xticks(rotation=45)
            tipus = "datetime"
        else:
            freq = df[col].value_counts().head(10)
            sns.barplot(x=freq.index.astype(str), y=freq.values)
            plt.title(f"Valors més freqüents de {col}")
            plt.xticks(rotation=45)
            tipus= "categoric"

        plt.tight_layout()
        plt.savefig(f"graficsS10/{col}_{tipus}.png")
        plt.close()
    
generar_grafics(df)

df_penguins = sns.load_dataset("penguins")
generar_grafics(df_penguins)

df_iris = sns.load_dataset("iris")
generar_grafics(df_iris)

df_titanic = sns.load_dataset("titanic")
generar_grafics(df_titanic)

In [61]:
df = pd.read_excel("matriu_distancies.xlsx", index_col=0)

df = df.drop(index=["Las Palmas de Gran Canaria", "Palma"], columns=["Las Palmas de Gran Canaria", "Palma"])

def ruta_mes_propera(matriu, ciutat_inicial):
    ciutats = list(matriu.index)
    
    visitades = [ciutat_inicial]
    no_visitades = set(ciutats)
    no_visitades.remove(ciutat_inicial)
    
    distancia_total = 0
    ciutat_actual = ciutat_inicial

    while no_visitades:
        distancies = matriu.loc[ciutat_actual, list(no_visitades)]
        
        ciutat_seguent = distancies.idxmin()
        distancia = distancies.min()
        distancia_total += distancia
        visitades.append(ciutat_seguent)
        no_visitades.remove(ciutat_seguent)
        ciutat_actual = ciutat_seguent

    return visitades, distancia_total

ruta, distancia = ruta_mes_propera(df, "Valencia")

print("Ruta:")
print(" - ".join(ruta))
print(f"Distància total recorreguda: {distancia:.0f} Km")

millor_ciutat = ''
millor_distancia = 10000
millor_ruta = ''

for ciutat in df.index:
    ruta, dist = ruta_mes_propera(df, ciutat)
    if dist < millor_distancia:
        millor_distancia = dist
        millor_ciutat = ciutat
        millor_ruta = ruta

print(f"\nMillor ciutat d'origen: {millor_ciutat}")
print(f"Distància total recorreguda: {millor_distancia:.0f} Km")

Ruta:
Valencia - Alicante - Murcia - Córdoba - Sevilla - Málaga - Valladolid - Gijón - Bilbao - Zaragoza - Hospitalet de Llobregat - Barcelona - Vigo
Distància total recorreguda: 3200 Km

Millor ciutat d'origen: Barcelona
Distància total recorreguda: 2646 Km
